* Model 8 is explored using DGX Station in NCCS
* Trainining Data is from DS 2
* Most accurate model, training for 299 epochs

In [1]:
# Import Libraries -------------------------------------------------------------------------------------------------------------------------------------------------------------
import os
import h5py
import keras
import loss
import Helper
import allMetrics
import numpy as np
import tensorflow as tf
import UNetModel_3D
import matplotlib.pyplot as plt
from keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, CSVLogger

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [ ]:
# /media/dro/JHSeagate/FYP



### Retrieving Dataset
* Input own path as necessary

In [5]:
dataset_num = 2
valid_dataset_num = 2

train_fileName = f"/media/dro/JHSeagate/FYP/jh_fyp_work/3D_UNet/trials/numpyDatasets/numPyArrays/train_Scans/train_DS{dataset_num}.hdf5"
train_maskfileName = f"/media/dro/JHSeagate/FYP/jh_fyp_work/3D_UNet/trials/numpyDatasets/numPyArrays/train_Mask_Scans/train_maskDS{dataset_num}.hdf5"

train_DatasetName = f"trainScans_DataSet{dataset_num}"
train_maskDatasetName = f"trainMaskScans_DataSet{dataset_num}"

valid_fileName = f"/media/dro/JHSeagate/FYP/jh_fyp_work/3D_UNet/trials/numpyDatasets/numPyArrays/valid_Scans/valid_DS{valid_dataset_num}.hdf5"
valid_maskfileName = f"/media/dro/JHSeagate/FYP/jh_fyp_work/3D_UNet/trials/numpyDatasets/numPyArrays/valid_Mask_Scans/valid_maskDS{valid_dataset_num}.hdf5"

valid_DatasetName = f"validScans_DataSet{valid_dataset_num}"
valid_maskDatasetName = f"validMaskScans_DataSet{valid_dataset_num}"

In [6]:
# Train Dataset
with h5py.File(train_fileName, 'r') as hf: # File Dir
    train_array = hf[train_DatasetName][:]
    
with h5py.File(train_maskfileName,'r') as hf:
    train_mask_array = hf[train_maskDatasetName][:]
    
with h5py.File(valid_fileName, 'r') as hf: # File Dir
    valid_array = hf[valid_DatasetName][:1350]
    
with h5py.File(valid_maskfileName,'r') as hf:
    valid_mask_array = hf[valid_maskDatasetName][:1350]

In [7]:
print(train_array.shape)
print(train_mask_array.shape)
print('--')
print(valid_array.shape)
print(valid_mask_array.shape)

(7875, 64, 64, 96)
(7875, 64, 64, 96)
--
(1350, 64, 64, 96)
(1350, 64, 64, 96)


In [8]:
train_array = np.expand_dims(train_array, axis=4)
train_mask_array = np.expand_dims(train_mask_array, axis=4)

valid_array = np.expand_dims(valid_array, axis=4)
valid_mask_array = np.expand_dims(valid_mask_array, axis=4)

In [9]:
print(train_array.shape)
print(train_mask_array.shape)
print('--')
print(valid_array.shape)
print(valid_mask_array.shape)

(7875, 64, 64, 96, 1)
(7875, 64, 64, 96, 1)
--
(1350, 64, 64, 96, 1)
(1350, 64, 64, 96, 1)


In [10]:
LR = 0.001
opt = tf.keras.optimizers.Nadam(LR)

input_shape = (64,64,96,1)
num_class = 1

metrics = [allMetrics.dice_coef]

model = UNetModel_3D.build_unet(input_shape, n_classes = num_class)
model.compile(optimizer=opt, loss=loss.tversky_crossentropy, metrics=metrics)
print(model.summary())

2023-03-02 14:49:15.298982: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-02 14:49:15.773597: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38414 MB memory:  -> device: 0, name: A100-SXM4-40GB, pci bus id: 0000:c2:00.0, compute capability: 8.0


sigmoid
Model: "U-Net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 96,  0           []                               
                                 1)]                                                              
                                                                                                  
 conv3d (Conv3D)                (None, 64, 64, 96,   896         ['input_1[0][0]']                
                                32)                                                               
                                                                                                  
 batch_normalization (BatchNorm  (None, 64, 64, 96,   128        ['conv3d[0][0]']                 
 alization)                     32)                                                   

In [ ]:
# class CustomCallBack(keras.callbacks.Callback):
#     def on_epoch_end(self, epoch, logs = None):
#         lr = self.model.optimizer.lr
#         decay = self.model.optimizer.decay
#         iterations = self.model.optimizer.iterations
#         lr_with_decay = lr / (1. + decay * K.cast(iterations, K.dtype(decay)))

#         Helper.telegram_bot_sendtext(f'''
#                             End of Epoch {epoch}
#                             Train Dice: {logs.get("dice_coef")}
#                             Valid Dice: {logs.get("val_dice_coef")}
#                             Train Loss: {logs.get("loss")}
#                             Val Loss: {logs.get('val_loss')}
#                             LR: {lr_with_decay}
#                               ''')

In [11]:
# /home/tester/jianhoong # ntu
# /media/dro/JHSeagate/FYP/ # dgx

csv_path = '/media/dro/JHSeagate/FYP/jh_fyp_work/3D_UNet/Final/CSVLogs/Model8_run2.csv'
model_checkpoint_path = '/media/dro/JHSeagate/FYP/jh_fyp_work/3D_UNet/Final/SavedModels/Model8_run2.hdf5'

In [12]:
my_callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, mode = 'auto'),
    EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True, mode = 'auto'),
    CSVLogger(csv_path, separator=',', append=True),
    ModelCheckpoint(filepath=model_checkpoint_path,
                    monitor='val_loss',
                    mode='auto',
                    verbose=1,
                    save_best_only= True)
]
#     CustomCallBack()

In [13]:
model_name = 'Model8 (300 Epochs)'
# Helper.telegram_bot_sendtext(f'Model {model_name} started training')

history = model.fit(train_array,
                    train_mask_array,
                    batch_size=5,
                    epochs=300,
                    verbose=1,
                    shuffle = True,
                    validation_data=(valid_array, valid_mask_array),
                    callbacks=my_callbacks)

Epoch 1/300


2023-03-02 14:49:49.627966: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8301
2023-03-02 14:49:50.486748: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 11.0.221, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


1575/1575 [==============================] - ETA: 0s - loss: 1.0179 - dice_coef: 0.0096
Epoch 00001: val_loss improved from inf to 1.02618, saving model to /media/dro/JHSeagate/FYP/jh_fyp_work/3D_UNet/Final/SavedModels/Model8_run2.hdf5
1575/1575 [==============================] - 265s 162ms/step - loss: 1.0179 - dice_coef: 0.0096 - val_loss: 1.0262 - val_dice_coef: 0.0041 - lr: 0.0010
Epoch 2/300
1575/1575 [==============================] - ETA: 0s - loss: 1.0086 - dice_coef: 0.0198
Epoch 00002: val_loss improved from 1.02618 to 1.02480, saving model to /media/dro/JHSeagate/FYP/jh_fyp_work/3D_UNet/Final/SavedModels/Model8_run2.hdf5
1575/1575 [==============================] - 250s 159ms/step - loss: 1.0086 - dice_coef: 0.0198 - val_loss: 1.0248 - val_dice_coef: 0.0049 - lr: 0.0010
Epoch 3/300
1575/1575 [==============================] - ETA: 0s - loss: 0.9986 - dice_coef: 0.0483
Epoch 00003: val_loss improved from 1.02480 to 1.02174, saving model to /media/dro/JHSeagate/FYP/jh_fyp_work

1575/1575 [==============================] - 250s 159ms/step - loss: 0.9589 - dice_coef: 0.1907 - val_loss: 0.9610 - val_dice_coef: 0.1211 - lr: 1.0000e-04
Epoch 25/300
1575/1575 [==============================] - ETA: 0s - loss: 0.9584 - dice_coef: 0.1985
Epoch 00025: val_loss did not improve from 0.96104
1575/1575 [==============================] - 248s 158ms/step - loss: 0.9584 - dice_coef: 0.1985 - val_loss: 0.9649 - val_dice_coef: 0.1123 - lr: 1.0000e-04
Epoch 26/300
1575/1575 [==============================] - ETA: 0s - loss: 0.9590 - dice_coef: 0.1952
Epoch 00026: val_loss did not improve from 0.96104
1575/1575 [==============================] - 248s 158ms/step - loss: 0.9590 - dice_coef: 0.1952 - val_loss: 0.9664 - val_dice_coef: 0.1187 - lr: 1.0000e-04
Epoch 27/300
1575/1575 [==============================] - ETA: 0s - loss: 0.9581 - dice_coef: 0.2057
Epoch 00027: val_loss did not improve from 0.96104
1575/1575 [==============================] - 248s 157ms/step - loss: 0.9581 

KeyboardInterrupt: 

In [ ]:
#plot the training and validation IoU and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

acc = history.history['dice_coef']
val_acc = history.history['val_dice_coef']

plt.plot(epochs, acc, 'y', label='Training Dice')
plt.plot(epochs, val_acc, 'r', label='Validation Dice')
plt.title('Training and validation Dice')
plt.xlabel('Epochs')
plt.ylabel('Dice')
plt.legend()
plt.show()